## Task 1: Data Handling

In [1]:
import pandas as pd
import numpy as np
import pickle
import os

import seaborn as sns
import matplotlib.pyplot as plt


# processing and measures
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer

# models
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor

import warnings
warnings.filterwarnings("ignore")
sns.set()

In [2]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
BASE_DIR = '/content/drive/MyDrive/CMP7005-1/beijing+multi+site+air+quality+data/PRSA2017_Data_20130301-20170228/PRSA_Data_20130301-20170228'

In [ ]:
#BASE_DIR = "beijing+multi+site+air+quality+data/PRSA2017_Data_20130301-20170228/PRSA_Data_20130301-20170228"

In [ ]:
# check the files available
os.listdir(BASE_DIR)

In [ ]:
# check the files size for each
for each_file_name in os.listdir(BASE_DIR):
    curr_ = pd.read_csv(f"{BASE_DIR}/{each_file_name}")
    file_name = each_file_name.split("_")[2]
    print(f"Data of {file_name} has a size {curr_.shape}")

### Data and Merging Process
- From above, the data consists of hourly air-quality data collected from 12 different monitoring sites in Beijing, spanning from March 1st, 2013 to February 28th, 2017. Each CSV file contains 18 columns, with data on various pollutants, meteorological conditions, and site-specific information.

- To merge these datasets, we first load each CSV and append each dataframe to a list. Then, we concatenate all the individual dataframes into a single dataframe, which combines the data along the rows while resetting the index to maintain a continuous numbering.
- This help have a merged data which is ready for exporation and analysis.

In [ ]:
df_lists = []

for each_file_name in os.listdir(BASE_DIR):
    curr_df_ = pd.read_csv(f"{BASE_DIR}/{each_file_name}")
    df_lists.append(curr_df_)
print("Completed Loading the datasets")

In [ ]:
#

In [ ]:
# merge the dataset into a single df
df = pd.concat(df_lists).reset_index(drop=True)

In [ ]:
# save the data for later usage on app
df.to_csv("merged_data.csv", index=False)

## Task 2: Exploratory Data Analysis (EDA)
- This part will involve;
    - Data pre-checkup
    - Exporation
    - Data Analysis and EDA.

### a) Fundamental data understanding

In [ ]:
# check sample data
df.sample(5)

In [ ]:
df.columns

In [ ]:
# check the number of records
df.info()

In [ ]:
# check statistics information
df.describe().T

In [ ]:
# check the number of columns
print(f"The Data has {df.shape[0]} total records and {df.shape[1]} columns ")

In [ ]:
# check columns with missing data
df.isna().sum()

In [ ]:
# what percentage of data is missing on each of the records
df.isna().sum()/df.shape[0]

In [ ]:
# we have two columns with non-numerical, what are their values

# get to see columns which are non numerical
df.select_dtypes(include=["object"])

In [ ]:
# check unique counts
# for WD
df["wd"].value_counts()

In [ ]:
# stations
df["station"].value_counts()

### Data Understanding Results
- The dataset includes records from 12 stations, each with 35,064 entries, resulting in a total of 420,768 records across all stations.
- The dataset contains 420,768 total records with 18 columns. These provides a detailed air quality and meteorological data collected from 12 monitoring stations in Beijing from March 1, 2013, to February 28, 2017.
- The columns in the dataset are a mix of pollutant concentrations like PM2.5, PM10, SO2, NO2, CO, O3 and also weather parameters liek  temperature, pressure, humidity, wind direction, and station identifiers.
- Each each row presents a hourly record of the data collected.
- All columns are numerical (consist of float and integer) except for `wind direction (wd)` and `station`


Below is a a summary of the dataset columns:

| Column   | Description                                      |
|----------|--------------------------------------------------|
| `No`     | Unique record identifier                         |
| `year`   | Year of the data entry                           |
| `month`  | Month of the data entry                          |
| `day`    | Day of the data entry                            |
| `hour`   | Hour of the data entry                           |
| `PM2.5`  | PM2.5 concentration (µg/m³)                      |
| `PM10`   | PM10 concentration (µg/m³)                       |
| `SO2`    | Sulfur dioxide concentration (ppb)               |
| `NO2`    | Nitrogen dioxide concentration (ppb)             |
| `CO`     | Carbon monoxide concentration (ppb)              |
| `O3`     | Ozone concentration (ppb)                        |
| `TEMP`   | Temperature (°C)                                  |
| `PRES`   | Atmospheric pressure (hPa)                       |
| `DEWP`   | Dew point temperature (°C)                        |
| `RAIN`   | Precipitation (mm)                               |
| `wd`     | Wind direction                                   |
| `WSPM`   | Wind speed (m/s)                                  |
| `station`| The monitoring station name (12 total)           |


### Missing Data Overview:
- **Missing Values**: The `PM2.5`, `PM10`, `SO2`, `NO2`, `CO`, `O3` columns have missing values, with `CO` having the highest number of missing entries (20,701).
- **Wind Direction (`wd`)**: This column has significant missing data, with 18% of values missing.
- The remaining columns like `TEMP`, `PRES`, `DEWP`, `RAIN`, and `WSPM` also have some missing data, but the count is relatively low compared to pollutants or wind data.


- In summary the dataset has a comprehensive view of air quality and meteorological conditions in Beijing over a 4-year period, with hourly records across 12 stations. There are some missing values, particularly in pollutant data and wind direction, but the dataset is mostly complete. This information is crucial for further analysis, such as investigating pollution trends, correlations between pollutants and weather factors, and making predictions or recommendations for improving air quality.

### b) Data preprocessing:
- This part will majory focused on dealing with missing and feature engineering.


-
- To prepare the dataset for analysis, several data processing steps were performed.
    - First, missing values in the numerical columns were imputed with the mean of their respective columns in order to ensure that no data was lost during the analysis. Also, missing values in the `wd` (wind direction) column were filled using forward fill (`ffill`), which propagates the last valid observation forward to fill missing entries
    - For feature engineering, a new `datetime` column was created by combining the `year`, `month`, and `day` columns into a single datetime object, which facilitates time-based analysis.
    - Finally, the `No` column, which was only an identifier for each file and not a relevant feature for analysis, was dropped from the dataset to reduce unnecessary data and improve the focus of the analysis.
-

These preprocessing steps ensured that the data was clean and consistent.

In [ ]:
df.head()

In [ ]:
# keep a copy of data
df_ = df.copy()

In [ ]:
# check any duplicated entries
print(f"There are  {df.duplicated().sum()} duplicates in the dataset")

In [ ]:
# The No. column is not a feature rather was identify for each file, we drop it
df.drop(columns = ["No"], inplace=True)

In [ ]:
df[df.isna().sum(axis=1)>0]

In [ ]:
# check if there is large different between means and medium in order to fill nulls
df.select_dtypes(exclude=["object"]).mean() - df.select_dtypes(exclude=["object"]).median()

In [ ]:
df[df.select_dtypes(exclude=["object"]).columns] = df.select_dtypes(exclude=["object"]).fillna(df.select_dtypes(exclude=["object"]).mean())

In [ ]:
# Fill missing using Forward fill
df["wd"]  = df["wd"].fillna(method='ffill')

In [ ]:
# check if there are other missing data
df.isna().sum().any()

In [ ]:
# Feature Engineering through combining year, month, day, and hour into a datetime column
df['datetime'] = pd.to_datetime(df[['year', 'month', 'day']])

df.head(2)

### c) Statistics/computation-based analysis and Visualisation:

- The major focus for exploratory data analysis (EDA) will be to gain insights into the seasonal variations and concentration levels of various air pollutants, such as PM2.5, PM10, NO2, and CO, across different times of the year.
- This part will analyze how these pollutants fluctuate over seasons, months, and hours of the day, uncovering any trends or patterns. - Also, we will compare the air temperature with pollutant concentrations to explore potential correlations between these conditions and air quality.
- This part will help provide a clearer picture on how air quality in Beijing is influenced by both seasonal changes and environmental conditions.

In [ ]:
# create a copy of data to use
data = df.copy()

In [ ]:
data.head()

####  Q: What was the wind Direction with most hourly records count.

In [ ]:
# check where the wind direction was mostly observed
wind_count_data= data['wd'].value_counts().sort_values(ascending=False)
# a bar chart show the results
plt.figure(figsize=(10, 6))
sns.barplot(x=wind_count_data.index, y=wind_count_data.values, palette='coolwarm')
plt.title('Top Wind Direction By Hourly Records', fontsize=16, fontweight='bold')
plt.xlabel('Wind Direction')
plt.ylabel('Hourly Count for data Provided')
plt.show()

##### Observation
- North Eastern is direction with most winds while West had the least.
- From the bar graph above, it can be clearly be seen that the Eastern and Nothern part are the directions with more counts than Western and Southen parts of Beijing.

#### Observation of how different features are distributed.

In [ ]:
def plot_data_distribution(data, category='AIR'):
    """
    Parameters:
    - data (pd.DataFrame)
    - category (str): either ('AIR' or 'WEATHER').
                      'AIR' plots air quality components,
                      'WEATHER' plots weather-related parameters (e.g., precipitation, temperature).
    """
    if category == 'AIR':
        variables = ['NO2', 'O3', 'SO2']
        category_title = "Air Quality Components"
        super_title = "Distribution and Boxplots of Key Air Quality Components (NO2, O3, SO2)"
    elif category == 'WEATHER':
        variables = ['TEMP', 'PRES', 'RAIN']
        category_title = "Weather Parameters"
        super_title = "Distribution and Boxplots of Key Weather Parameters (Temperature, Pressure, Precipitation)"
    else:
        raise ValueError("Invalid category. Choose either 'AIR' or 'WEATHER'.")


    fig, axes = plt.subplots(2, 3, figsize=(18, 9))

    for i, feature in enumerate(variables):
        row = i // 3
        col = i % 3

        sns.histplot(data[feature], kde=True, bins=20, color='skyblue', ax=axes[row, col])
        axes[row, col].set_title(f'Distribution of {feature} ({category_title})', fontsize=14, fontweight='bold')
        axes[row, col].set_xlabel(feature)
        axes[row, col].set_ylabel('Frequency')
        sns.boxplot(data[feature], orient='h', ax=axes[row+1, col] if row+1 < 2 else axes[row, col])
        axes[row+1, col].set_title(f'Boxplot of {feature} ({category_title})', fontsize=14, fontweight='bold')
        axes[row+1, col].set_xlabel(feature)
    plt.suptitle(super_title, fontsize=18, fontweight='bold', ha='center', y=1.02)
    plt.tight_layout()
    plt.show()


In [ ]:
#NO2, O3, SO2
plot_data_distribution(data, category='AIR')


In [ ]:
#TEMP, PRES, RAIN)
plot_data_distribution(data, category='WEATHER')


##### Observation
Interms of distribution, the following can be observed for each case;
   - The distribution of temperature likely shows a **bimodal which is also near-normal distribution**. This is becuase of different seasonal variation in Beijing where the peak are the warmer periods. As from the box plot, it can be observed that the distribution is centered around the mean with no outliers. It's mean is aound `14 degrees`
   - The pressure distribution is ** near symmetric**, with small change around a mean value. There are no outlier identified and its mean is around `1010`.
   - The rainfall distribution is very **rightly-skewed**, with most data points showing little or no rainfall and a smaller number of days with heavy rainfall. This hence cause it to show multiple outliers due to large deviation in values.
   - The distribution of NO2 is **right-skewed**, showing higher concentrations during winter months due to traffic and heating, and lower levels during warmer months. It has fewer outliers which might be due to instances of  heavy pollution events.
   - The O3/Ozone distribution has a **rightly skewed**, and has a very large variation hence more outliers.
   - SO2 is **right-skewed**, with a majority of days showing low concentrations, but some outliers indicating pollution spikes, often linked to industrial activities or certain weather conditions.


#### Line Plot for Monthly Trends
- In this case we try to observe the concentration of pollutants like PM2.5, NO2, and O3 change over the months, which helps highlight seasonal trends.

In [ ]:
# since we have different years, we can create a variable containing year and month informtion
data['year_month'] = data['year'].astype(str) + '-' + data['month'].astype(str).str.zfill(2)

# lets plot AIR Information: PM2.5, NO2, O3, So2 for monthly trends
plt.figure(figsize=(15, 14))

plt.subplot(5, 1, 1)
monthly_pm25 = data.groupby('year_month')['PM2.5'].mean()
monthly_pm25.plot(kind='line', color='skyblue', marker='o')
plt.title('Monthly Average PM2.5 Concentration', fontsize=16, fontweight='bold')
plt.xlabel('Month', fontsize=12)
plt.ylabel('PM2.5 (µg/m³)', fontsize=12)

plt.subplot(5, 1, 2)
monthly_no2 = data.groupby('year_month')['NO2'].mean()
monthly_no2.plot(kind='line', color='lightgreen', marker='o')
plt.title('Monthly Average NO2 Concentration', fontsize=16, fontweight='bold')
plt.xlabel('Month', fontsize=12)
plt.ylabel('NO2 (µg/m³)', fontsize=12)

plt.subplot(5, 1, 3)
monthly_o3 = data.groupby('year_month')['O3'].mean()
monthly_o3.plot(kind='line', color='lightcoral', marker='o')
plt.title('Monthly Average O3 Concentration', fontsize=16, fontweight='bold')
plt.xlabel('Month', fontsize=12)
plt.ylabel('O3 (µg/m³)', fontsize=12)


plt.subplot(5, 1, 4)
monthly_o3 = data.groupby('year_month')['SO2'].mean()
monthly_o3.plot(kind='line', color='red', marker='o')
plt.title('Monthly Average SO2 Concentration', fontsize=16, fontweight='bold')
plt.xlabel('Month', fontsize=12)
plt.ylabel('SO2 (µg/m³)', fontsize=12)


plt.subplot(5, 1, 5)
monthly_o3 = data.groupby('year_month')['CO'].mean()
monthly_o3.plot(kind='line', color='purple', marker='o')
plt.title('Monthly Average CO Concentration', fontsize=16, fontweight='bold')
plt.xlabel('Month', fontsize=12)
plt.ylabel('CO (µg/m³)', fontsize=12)


plt.tight_layout()
plt.show()


##### Observations
- In above graphs, for `PM2.5, SO2 and NO2`, it can be observed that there is an increase in PM2.5, SO2 and NO2 concentrations during winter months (November to February). THe cause might be due to more increased heat emmissions and also vehicles used during the cold weather. This leads to higher pollutant levels in the atmosphere.
- In above, the `Ozone (O3)` shows higher levels in the warmer months (May to August). It seems since O3 is influenced by sunlight, it peaks on warmer months when sunlight and temperatures are higher.
- `CO` is observed to have a similar trend to that of `NO2`
- It can also be observed that when `PM2.5, SO2 and NO2`, we obseve `O3` having low points and vice versa.

##### Line plot and  Boxplot for Hourly Trends
In this case a plot plot is used to show how pollutant levels vary by hour of the day. THis helps to understand any diurnal (daily) patterns, such as higher pollution during rush hours.


In [ ]:
colors = ['red','green', 'orange', 'purple', 'yellow', 'blue', 'cyan', 'magenta', 'gray', 'brown']

In [ ]:
for i, column in enumerate(["PM2.5", "PM10", "NO2", "O3", "SO2", 'TEMP','PRES','DEWP', "WSPM"]):
    data_hr = data[[column,'hour']].groupby(["hour"]).median().reset_index().sort_values(by='hour',ascending=False)
    f,ax=plt.subplots(figsize=(15,5))
    sns.pointplot(x='hour', y=str(column), data=data_hr, markers='o', color=colors[i % len(colors)])
    plt.title(f"Hourly Analysis for {column}", fontweight='bold', fontsize=17)
    plt.xlabel("Hour",  fontweight='bold', fontsize=15)
    plt.ylabel("PM2.5 Values", fontweight='bold', fontsize=17)

In [ ]:
# Plot hourly variation for 5 items accross the whole data
plt.figure(figsize=(18, 14))

plt.subplot(5, 1, 1)
sns.boxplot(x='hour', y='PM2.5', data=data, palette='Blues', showfliers=False)
plt.title('Hourly PM2.5 Concentration', fontsize=16, fontweight='bold')
plt.xlabel('Hour of the Day', fontsize=12)
plt.ylabel('PM2.5 (µg/m³)', fontsize=12)

plt.subplot(5, 1, 2)
sns.boxplot(x='hour', y='NO2', data=data, palette='Greens', showfliers=False)
plt.title('Hourly NO2 Concentration', fontsize=16, fontweight='bold')
plt.xlabel('Hour of the Day', fontsize=12)
plt.ylabel('NO2 (µg/m³)', fontsize=12)

plt.subplot(5, 1, 3)
sns.boxplot(x='hour', y='O3', data=data, palette='Reds', showfliers=False)
plt.title('Hourly O3 Concentration', fontsize=16, fontweight='bold')
plt.xlabel('Hour of the Day', fontsize=12)
plt.ylabel('O3 (µg/m³)', fontsize=12)

plt.subplot(5, 1, 4)
sns.boxplot(x='hour', y='SO2', data=data, palette='Purples', showfliers=False)
plt.title('Hourly  SO2 Concentration', fontsize=16, fontweight='bold')
plt.xlabel('Hour of the Day', fontsize=12)
plt.ylabel('SO2 (µg/m³)', fontsize=12)


plt.subplot(5, 1, 5)
sns.boxplot(x='hour', y='CO', data=data, palette='rocket', showfliers=False)
plt.title('Hourly  CO Concentration', fontsize=16, fontweight='bold')
plt.xlabel('Hour of the Day', fontsize=12)
plt.ylabel('CO (µg/m³)', fontsize=12)



plt.tight_layout()
plt.show()


###### Observations:
- `PM2.5, CO, and NO2` is observed above that there are some higher pollution levels during peak hours which include 7 AM to 9 AM and 5 PM to 7 PM). This can be associated with increased motor emissions during morning and evening rush hours.
- `SO2` is seen to be up from 7AM upto about 6PM which is normally the normal working days.
- `PM2.5` levels might be high during the early morning and evening when traffic congestion is at its peak.
- `Ozone (O3)` have higher levels between 11AM upto about 7PM  with very high level in the afternoon (1Pm to 6PM). This is likely to be more in sunny day, as ozone forms from reactions between NO2 emissions and sunlight.

### Heatmap for Hourly and Monthly Trends
- In this heatmap is gonna be used to view summary of how pollutant levels (e.g., PM2.5, NO2, O3) vary across both the month and hour of the day. It helps to see if there are patterns influenced by both the time of the day or seasonal factors.

In [ ]:

# plot a heatmap for hourly and monthly trends of air quality pollutants.

# create pivottnfor data to get the average pollutant level per month and hour
pm25_pivot = data.pivot_table(values='PM2.5', index='hour', columns='month', aggfunc=np.mean)
no2_pivot = data.pivot_table(values='NO2', index='hour', columns='month', aggfunc=np.mean)
o3_pivot = data.pivot_table(values='O3', index='hour', columns='month', aggfunc=np.mean)
so2_pivot = data.pivot_table(values='SO2', index='hour', columns='month', aggfunc=np.mean)
co_pivot = data.pivot_table(values='CO', index='hour', columns='month', aggfunc=np.mean)


plt.figure(figsize=(18, 15))

plt.subplot(5, 1, 1)
sns.heatmap(pm25_pivot, cmap='Blues', annot=False, cbar_kws={'label': 'PM2.5 (µg/m³)'})
plt.title('Heatmap of PM2.5 by Hour and Month', fontsize=16, fontweight='bold')
plt.xlabel('Month', fontsize=12)
plt.ylabel('Hour of the Day', fontsize=12)

plt.subplot(5, 1, 2)
sns.heatmap(no2_pivot, cmap='Greens', annot=False, cbar_kws={'label': 'NO2 (µg/m³)'})
plt.title('Heatmap of NO2 by Hour and Month', fontsize=16, fontweight='bold')
plt.xlabel('Month', fontsize=12)
plt.ylabel('Hour of the Day', fontsize=12)

plt.subplot(5, 1, 3)
sns.heatmap(o3_pivot, cmap='Reds', annot=False, cbar_kws={'label': 'O3 (µg/m³)'})
plt.title('Heatmap of O3 by Hour and Month', fontsize=16, fontweight='bold')
plt.xlabel('Month', fontsize=12)
plt.ylabel('Hour of the Day', fontsize=12)


plt.subplot(5, 1, 4)
sns.heatmap(so2_pivot, cmap='rocket_r', annot=False, cbar_kws={'label': 'SO2 (µg/m³)'})
plt.title('Heatmap of SO2 by Hour and Month', fontsize=16, fontweight='bold')
plt.xlabel('Month', fontsize=12)
plt.ylabel('Hour of the Day', fontsize=12)

plt.subplot(5, 1, 5)
sns.heatmap(co_pivot, cmap='mako_r', annot=False, cbar_kws={'label': 'CO (µg/m³)'})
plt.title('Heatmap of CO by Hour and Month', fontsize=16, fontweight='bold')
plt.xlabel('Month', fontsize=12)
plt.ylabel('Hour of the Day', fontsize=12)


plt.tight_layout()
plt.show()



##### Observations
- In above, it is observed that `PM2.5, CO and NO2` Heatmaps shows higher pollutant levels during winter months i.e December, January, and February though also october and novermber and also during peak traffic hours i.e 7-9 AM and 5-7 PM. This is consistent with the increased emissions from heating and vehicles in colder months.
- For `Ozone (O3)`, the levels shows a spike during April to August (which are generally warmer periods) with more concentration between 12Pm to 6Pm. THis might be sun driven with chemical reactions.
- For `SO2`, there is seen to have higher concentration across January, Feb, March and Novermber and DEcember. The trend for SO2 is seen to be like during colder periods, it has higher values. It seems to be inversely related to `O3`






-

- **From above 3 graphs it can be summaried that PM2.5, NO2, CO and SO2 are likely to peak on the colders periods and during rush hours while O3 is likely to peak during high levels in warmer months especially during the midday houses**

##### Temperature (TEMP) vs. Air Quality
- Temperature can influence air pollutants concentrations , especially O3 (ozone), which forms under sunlight and higher temperatures.

In [ ]:
# plot temperature vs. air quality pollutants

plt.figure(figsize=(14, 8))

# Plot PM2.5 vs. Temperature
plt.subplot(2, 2, 1)
sns.scatterplot(x='TEMP', y='PM2.5', data=data, color='skyblue')
plt.title('PM2.5 vs Temperature', fontsize=16, fontweight='bold')
plt.xlabel('Temperature (°C)', fontsize=12)
plt.ylabel('PM2.5 (µg/m³)', fontsize=12)

# Plot NO2 vs. Temperature
plt.subplot(2, 2, 2)
sns.scatterplot(x='TEMP', y='NO2', data=data, color='lightgreen')
plt.title('NO2 vs Temperature', fontsize=16, fontweight='bold')
plt.xlabel('Temperature (°C)', fontsize=12)
plt.ylabel('NO2 (µg/m³)', fontsize=12)

# Plot O3 vs. Temperature
plt.subplot(2, 2, 3)
sns.scatterplot(x='TEMP', y='O3', data=data, color='lightcoral')
plt.title('O3 vs Temperature', fontsize=16, fontweight='bold')
plt.xlabel('Temperature (°C)', fontsize=12)
plt.ylabel('O3 (µg/m³)', fontsize=12)

# Plot SO2 vs. Temperature
plt.subplot(2, 2, 4)
sns.scatterplot(x='TEMP', y='SO2', data=data, color='yellow')
plt.title('SO2 vs Temperature', fontsize=16, fontweight='bold')
plt.xlabel('Temperature (°C)', fontsize=12)
plt.ylabel('SO2 (µg/m³)', fontsize=12)


plt.tight_layout()
plt.show()


##### Observations:
- For the case of `PM2.5, SO2 and NO2`, the temperature shows no direct correlation since these pollutants are more influenced by  emissions (mostly engines/motor emissions), heating (in colder months), and weather patterns rather than temperature alone.
- For the case of `O3`, it is seen that there appear some positive correlation between O3 and temperature because ozone is a secondary pollutant formed from reactions between NOx emissions and sunlight, which is more intense during warmer temperatures.

### 2. Rainfall (RAIN) vs. Air Quality
Rainfall can directly influence the levels of pollutants, as it can wash pollutants from the atmosphere, reducing concentrations.

In [ ]:
# plot rainfall vs. air quality pollutants

plt.figure(figsize=(14, 8))

# Plot PM2.5 vs. Rainfall
plt.subplot(2, 2, 1)
sns.scatterplot(x='RAIN', y='PM2.5', data=data, color='skyblue')
plt.title('PM2.5 vs Rainfall', fontsize=16, fontweight='bold')
plt.xlabel('Rainfall (mm)', fontsize=12)
plt.ylabel('PM2.5 (µg/m³)', fontsize=12)

# Plot NO2 vs. Rainfall
plt.subplot(2, 2, 2)
sns.scatterplot(x='RAIN', y='NO2', data=data, color='lightgreen')
plt.title('NO2 vs Rainfall', fontsize=16, fontweight='bold')
plt.xlabel('Rainfall (mm)', fontsize=12)
plt.ylabel('NO2 (µg/m³)', fontsize=12)

# Plot O3 vs. Rainfall
plt.subplot(2, 2, 3)
sns.scatterplot(x='RAIN', y='O3', data=data, color='lightcoral')
plt.title('O3 vs Rainfall', fontsize=16, fontweight='bold')
plt.xlabel('Rainfall (mm)', fontsize=12)
plt.ylabel('O3 (µg/m³)', fontsize=12)


# Plot CO vs. Rainfall
plt.subplot(2, 2, 4)
sns.scatterplot(x='RAIN', y='CO', data=data, color='red')
plt.title('CO vs Rainfall', fontsize=16, fontweight='bold')
plt.xlabel('Rainfall (mm)', fontsize=12)
plt.ylabel('CO (µg/m³)', fontsize=12)

plt.tight_layout()
plt.show()


##### Observations:
- It is observed that there is a negative correlation between rainfall and `PM2.5, CO and NO2.`. THe rain can wash pollutants from the atmosphere, resulting in lower concentrations during or after rainfall.
- Also the same thing is observed on `O3` as it has a negative correlation with rainfall. THis is becuase the rain helps clear ozone (O3) from the atmosphere.

##### Wind Speed (WSPM) and Direction (wd) vs. Air Quality
Wind speed and direction are crucial in understanding how pollutants disperse or accumulate. Higher wind speeds disperse pollutants, while lower wind speeds might result in pollutant accumulation.

In [ ]:
# plot wind speed vs. air quality pollutants.

plt.figure(figsize=(14, 8))

# Plot PM2.5 vs. Wind Speed
plt.subplot(2, 2, 1)
sns.scatterplot(x='WSPM', y='PM2.5', data=data, color='blue')
plt.title('PM2.5 vs Wind Speed', fontsize=16, fontweight='bold')
plt.xlabel('Wind Speed (m/s)', fontsize=12)
plt.ylabel('PM2.5 (µg/m³)', fontsize=12)

# Plot NO2 vs. Wind Speed
plt.subplot(2, 2, 2)
sns.scatterplot(x='WSPM', y='NO2', data=data, color='green')
plt.title('NO2 vs Wind Speed', fontsize=16, fontweight='bold')
plt.xlabel('Wind Speed (m/s)', fontsize=12)
plt.ylabel('NO2 (µg/m³)', fontsize=12)

# Plot O3 vs. Wind Speed
plt.subplot(2, 2, 3)
sns.scatterplot(x='WSPM', y='O3', data=data, color='red')
plt.title('O3 vs Wind Speed', fontsize=16, fontweight='bold')
plt.xlabel('Wind Speed (m/s)', fontsize=12)
plt.ylabel('O3 (µg/m³)', fontsize=12)

# Plot CO vs. Wind Speed
plt.subplot(2, 2, 4)
sns.scatterplot(x='WSPM', y='CO', data=data, color='purple')
plt.title('CO vs Wind Speed', fontsize=16, fontweight='bold')
plt.xlabel('Wind Speed (m/s)', fontsize=12)
plt.ylabel('CO (µg/m³)', fontsize=12)

plt.tight_layout()
plt.show()


##### Observation
- It is observed that higher wind speeds have a negative correlation with polutants. THis is because wind might help disperse the polutant away. For `O3`, the correlation is very small.
-

##### DEw Point vs. Air Quality

In [ ]:
# plot dew point vs. air quality pollutants.


plt.figure(figsize=(14, 8))

# Plot PM2.5 vs. Dew Point
plt.subplot(2, 2, 1)
sns.scatterplot(x='DEWP', y='PM2.5', data=data, color='purple')
plt.title('PM2.5 vs Dew Point', fontsize=16, fontweight='bold')
plt.xlabel('Dew Point (°C)', fontsize=12)
plt.ylabel('PM2.5 (µg/m³)', fontsize=12)

# Plot NO2 vs. Dew Point
plt.subplot(2, 2, 2)
sns.scatterplot(x='DEWP', y='NO2', data=data, color='yellow')
plt.title('NO2 vs Dew Point', fontsize=16, fontweight='bold')
plt.xlabel('Dew Point (°C)', fontsize=12)
plt.ylabel('NO2 (µg/m³)', fontsize=12)

# Plot O3 vs. Dew Point
plt.subplot(2, 2, 3)
sns.scatterplot(x='DEWP', y='O3', data=data, color='red')
plt.title('O3 vs Dew Point', fontsize=16, fontweight='bold')
plt.xlabel('Dew Point (°C)', fontsize=12)
plt.ylabel('O3 (µg/m³)', fontsize=12)

# Plot CO vs. Dew Point
plt.subplot(2, 2, 4)
sns.scatterplot(x='DEWP', y='CO', data=data, color='green')
plt.title('CO vs Dew Point', fontsize=16, fontweight='bold')
plt.xlabel('Dew Point (°C)', fontsize=12)
plt.ylabel('CO (µg/m³)', fontsize=12)

plt.tight_layout()
plt.show()


##### Observation
- The scatter plot  for `Pm2.5` shows a **weak correlation** between PM2.5 levels and dew point. We expect more dew points in colder times hence higher PM2.5 might coincide with higher humidity though the relationship is not strong.
- For the case of `NO2`, it has some **moderate correlation** between NO2 and dew point. High dew points may trap NO2 closer to the ground, increasing pollution.
- The scatter plot for Ozone (O3) is shows a **strong positive correlation** with dew point, especially during the summer months. Ozone forms in warm, sunny conditions, and higher dew points indicate high humidity, which can increases ozone formation.
- The relationship between CO and dew point may be **weak or non-linear**. CO is primarily linked to combustion processes while higher humidity can affect air quality, it’s not as directly tied to CO levels as it is for other pollutants like O3.


#### Correlation Analysis on Numerical columns

In [ ]:
# select numerical columns
# We exlude the TIME BASED VARIABLES
df_num = df.select_dtypes(include=["float", "int"]).drop(columns=["year", "month", "day", "hour"])
df_num.head(3)

In [ ]:
# plot corrrelation analysis
plt.figure(figsize=(15,15))
corr = df_num.corr()
sns.heatmap(corr, annot=True, cmap="YlGnBu", cbar=False)
plt.title("Correlation Among  Pollutant Related Features", fontsize=16, fontweight='bold')
plt.xticks(rotation=45, fontweight='bold')
plt.yticks(rotation=45, fontweight='bold')
plt.show()

##### Observation from correlation analysis
- **PM2.5 and PM10** are strongly correlated, and their levels are linked with common sources of pollution, such as industrial emissions and vehicular traffic.
- **CO**, **NO2**, and **SO2** tend to increase together, reflecting emissions from combustion processes.
- **Ozone** (O3) behaves differently, showing negative correlations with most primary pollutants, especially **NO2**.
- **Temperature** and **dew point** have a strong relationship, and both can influence pollutant behavior, with higher temperatures generally leading to lower concentrations of particulate matter.
- **Rainfall** and **wind speed** show weak to moderate impacts on air quality, with wind likely dispersing pollutants and rainfall having a temporary cleansing effect.

### TOp correlations with `PM2.5`

In [ ]:
# get correlation with PM2.5

# extract the PM2.5 column and sort the correlations
pm25_corr = corr['PM2.5'].abs().sort_values(ascending=False)
# remove the first since is stroke itself
top_corr = pm25_corr.index[1:]

# a bar chart show the results
plt.figure(figsize=(10, 6))
sns.barplot(x=pm25_corr[top_corr], y=top_corr, palette='rocket')
plt.title('Top Correlations with PM2.5', fontsize=16, fontweight='bold')
plt.xlabel('Absolute Correlation')
plt.ylabel('Features')

plt.show()

#### Observation
- PM2.5 shows **strong positive correlations** with **PM10 (0.88)**, **CO (0.77)**, and **NO2 (0.66)** which indicates that these pollutants often rise together, likely due to shared sources like traffic and industrial emissions.
- It has **moderate correlation** with **SO2 (0.48)**,
- The correlation with **wind speed (0.27)** is weak, suggesting that wind has a limited effect on PM2.5 concentrations.
- There’s a **weak positive correlation** with **O3 (0.15)**, likely due to overlapping conditions affecting both pollutants, and a very weak correlation with **temperature (0.13)**, indicating that temperature has little direct impact on PM2.5 levels.
- The correlations with **dew point (0.11)**, **pressure (0.02)**, and **rain (0.01)** are very weak, showing minimal influence on PM2.5 concentrations.

## Task 3: Model Building for Air Quality Prediction
- This task will involve building a machine learning model to predict air quality focusing on pollutant levels such as **PM2.5**, **NO2**, or **O3**, based on various environmental and meteorological features (e.g., temperature, pressure, wind speed, etc.). The process includes:
    1. **Data Preprocessing**: Clean and transform the data for model input.
    2. **Feature Selection**: Identify relevant features for the model.
    3. **Model Selection**: Choose an appropriate machine learning algorithm.
    4. **Model Evaluation**: Evaluate the model's performance using appropriate metrics.



- Below are the models which are going to be used with basic regression models used as baseline model;
    - **Linear Regression**
    - **Random Forest Regressor**
    - **XGBoost**
    - **Gradient Boosting Regressor**

- THe models will be evaluated using the following measures;
    - **Mean Absolute Error (MAE)**
    - **Mean Squared Error (MSE)**
    - **Root Mean Squared Error (RMSE)**
    - **R² (coefficient of determination)**

In [ ]:
# recheck the data
df.head()

In [ ]:
# encode categorical variables like 'station', 'wd' using LabelEncoder

# we have encoder for each one of them since we need to reuse for predictions
encoder_station = LabelEncoder()
encoder_wd = LabelEncoder()
df['station'] = encoder_station.fit_transform(df['station'])
df['wd'] = encoder_wd.fit_transform(df['wd'])


In [ ]:
df.head(3)

In [ ]:
df.shape

In [ ]:
# take a sample of data due to computation issues
df_sample = df.drop("datetime",axis=1).sample(frac=0.25, random_state=2024)

In [ ]:
# Feature selection - we will predict PM2.5
X = df_sample.drop(columns=['PM2.5'])
Y = df_sample['PM2.5']

In [ ]:
# Split the data into training and test sets
Xtrain, Xtest, Ytrain, Ytest = train_test_split(
    X, Y,
    test_size=0.2,
    random_state=42)

Xtrain.shape, Xtest.shape, Ytrain.shape, Ytest.shape

In [ ]:
# feature scaling (Standardization)
scaler = StandardScaler()
Xtrain_scaled = scaler.fit_transform(Xtrain)
Xtest_scaled = scaler.transform(Xtest)

In [ ]:
# create model objects
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
    'XGBoost': XGBRegressor(n_estimators=100, random_state=42)
}

In [ ]:
# train these models
results = {}
for model_name, model in models.items():
    model.fit(Xtrain_scaled, Ytrain)
    Ypred = model.predict(Xtest_scaled)

    mae = mean_absolute_error(Ytest, Ypred)
    mse = mean_squared_error(Ytest, Ypred)
    rmse = np.sqrt(mse)
    r2 = r2_score(Ytest, Ypred)

    results[model_name] = {
        'MAE': mae,
        'MSE': mse,
        'RMSE': rmse,
        'R2': r2
    }

In [ ]:
# show the perfomance results
perfomance_res = pd.DataFrame(results).T
perfomance_res

#### Residual Plots
- Residuals plots shows the differences between the actual and predicted values in a model.
- It helps in evaluating model performance and identify how well the model fits the data and whether any patterns remain unexplained. - - In a perfect case, residuals should be randomly distributed around zero, indicating that the model has captured the underlying relationships in the data. They wills provide insight into where the model is making errors, helping us refine and improve the model for better predictions.

In [ ]:
# ftn to plot residuals for each model
def plot_residuals(models, Xtest_scaled, Ytest, data):
    plt.figure(figsize=(14, 10))

    for i, (model_name, model) in enumerate(models.items(), 1):
        predicted = model.predict(Xtest_scaled)

        compare_data = pd.DataFrame({
            'dates': data.iloc[Ytest.index]["year_month"],
            'Actual PM2.5': Ytest,
            'Predicted PM2.5': predicted
        }).reset_index(drop=True)

        compare_data['Residuals'] = compare_data['Actual PM2.5'] - compare_data['Predicted PM2.5']

        # set date as index
        compare_data.set_index("dates", inplace=True)

        plt.subplot(2, 2, i)
        plt.scatter(compare_data.index, compare_data['Residuals'], alpha=0.7, color='b')
        plt.axhline(y=0, color='r', linestyle='--')
        plt.title(f'{model_name} Residuals', fontsize=16)
        plt.xlabel('Index', fontsize=12)
        plt.ylabel('Residuals', fontsize=12)
        plt.xticks([])
        plt.grid(True)

    plt.tight_layout()
    plt.show()

plot_residuals(models, Xtest_scaled, Ytest, data)


#### Hyperparameter Tuning
- For the best-performing model, In this case we will use `Gradient Boosting Regressor`, we will perform hyperparameter tuning using GridSearchCV to optimize the model.

In [ ]:
from sklearn.model_selection import GridSearchCV

tune_model = GradientBoostingRegressor(random_state=42)

# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 5],
    'subsample': [0.8,  1.0],
    'loss': ['ls', 'huber']
}

# start the grid seacher
grid_search = GridSearchCV(estimator=tune_model, param_grid=param_grid, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)

In [ ]:
# fit the model with the training data
grid_search.fit(Xtrain_scaled, Ytrain)


In [ ]:
# get the best results
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# print them
print(f"Best Parameters: {best_params}")
print(f"Best MAE: {-grid_search.best_score_}")


In [ ]:
y_pred = best_model.predict(Xtest_scaled)

mae = mean_absolute_error(Ytest, y_pred)
mse = mean_squared_error(Ytest, y_pred)
rmse = mean_squared_error(Ytest, y_pred, squared=False)
r2 = r2_score(Ytest, y_pred)

print(f"Tuned Model MAE: {mae}")
print(f"Tuned Model MSE: {mse}")
print(f"Tuned Model RMSE: {rmse}")
print(f"Tuned Model R2: {r2}")


In [ ]:
plot_residuals({"TUNED GRADIENT BOOSTING": best_model}, Xtest_scaled, Ytest, data)

##### Observation,
- It is observed, on optimization, the Gradient boosting perfomed very good moving from previous `MAE of about 15 to about 12 and RMSE from about 25 to about 20`. Its `R2 score also increased by 3 points`

In [ ]:
# save the best model to be used for predictions
with open('gui_app/models/best_gradient_model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [ ]:
# also save scalers and encoders

with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)


with open('encoder_wd.pkl', 'wb') as f:
    pickle.dump(encoder_wd, f)


with open('encoder_station.pkl', 'wb') as f:
    pickle.dump(encoder_station, f)

In [ ]:
# also save the data
data.to_csv("cleaned_data.csv", index=False)

In [ ]:
def predict_air_quality(user_input, model, scaler, encoder_wd, encoder_station):
    """
    Function to make predictions on air quality data based on user input.

    Parameters:
    user_input (dict): A dictionary with keys as the feature names and values as the feature data.
    model (sklearn model): The trained model (e.g., a regressor or classifier).
    scaler (sklearn transformer): The fitted scaler used for scaling the input data.
    encoder_wd (sklearn encoder): The fitted encoder for the 'wd' (wind direction) feature.
    encoder_station (sklearn encoder): The fitted encoder for the 'station' feature.

    Returns:
    float: The predicted value (e.g., PM2.5, PM10, etc.) based on the model.
    """

    if user_input["wd"] not in encoder_wd.classes_:
        return False, f"YOur WD Input {user_input['wd']} Is incorrect, available wind directions are {list(encoder_wd.classes_)}"


    if user_input["station"] not in encoder_station.classes_:
        return False, f"YOur Station Input {user_input['wd']} Is incorrect, available wind directions are {list(encoder_station.classes_)}"

    # convert user input dictionary to DataFrame
    user_df = pd.DataFrame([user_input])

    # encode categorical features (wind direction and station)
    user_df['wd'] = encoder_wd.transform(user_df['wd'].values.reshape(-1, 1))
    user_df['station'] = encoder_station.transform(user_df['station'].values.reshape(-1, 1))

    # Scaling the features
    scaled_data = scaler.transform(user_df[['year', 'month', 'day', 'hour', 'PM10', 'SO2', 'NO2', 'CO',
       'O3', 'TEMP', 'PRES', 'DEWP', 'RAIN', 'wd', 'WSPM', 'station']])

    # Make prediction
    prediction = model.predict(scaled_data)

    return True, prediction[0]


In [ ]:
# Example of user input
user_input = {
    'year': 2013,
    'month': 3,
    'day': 1,
    'hour': 0,
    'PM10': 9.0,
    'SO2': 3.0,
    'NO2': 17.0,
    'CO': 300.0,
    'O3': 89.0,
    'TEMP': -0.5,
    'PRES': 1024.5,
    'DEWP': -21.4,
    'RAIN': 0.0,
    'wd': 'SSW',
    'WSPM': 5.7,
    'station': 'Dongsi',
}

prediction = predict_air_quality(user_input, model, scaler, encoder_wd, encoder_station)

if not prediction[0]:
    print(prediction[1])
else:
    print(f"Predicted PM2.5: {prediction[1]}")
